<div style="text-align: center; color: blue">
    <h1>A TIME SERIES MODEL</h1>
    <p style="text-align: center; font-size: 20px; color: Orange">Forecasting real estate prices of various zip codes and determining the top five (5) best zipcodes for Waridi Investments to venture into. 
    </p>
</div>



## Group 8 Project Colaborators

**STUDENT NAMES:**  
+ SAMMY SIFUNA  
+ JULIUS CHARLES  
+ WARUCH KURIA  
+ RAEL NDONYE  
+ ALLAN OMONDI  
+ JANET KHAINZA

## Business Understanding
       
### Background
Between `1996` to `2018`, the U.S. real estate market underwent significant transformations. It commenced with an increase in home prices driven by a growing perception of real estate as a valuable long-term investment. However, the Great Recession between `2007` to `2009` brought about extensive declines in home values, triggering foreclosures and eroding trust.
Over time,low interest rates and government interventions helped restore stability and confidence in the market. Urbanization gained momentum as more people gravitated towards cities, younger individuals entered the housing market, and technology assumed a pivotal role. Although certain cities thrived, others faced challenges. Regulatory reforms were introduced to prevent potential crises. Hence, the era marked a period of adaptation and evolution in the dynamics of buying and selling homes.
![Depiction of a Changing Market Prices](images/time-series-analysis.jpg)


### Problem Statement
Waridi Investments, a recently established real estate investment firm, has engaged our services to identify the top `5 zip codes` with the potential for the highest return on investment when they `sell in 5 years`. Their strategic approach is to initiate short-term investments in one of the most thriving real estate markets in the United States of America. The company places significant emphasis on securing sound investments that ensure consistent cash flow, ultimately enabling them to reinvest effectively when the opportune moment arises.

### Objectives
* To determine the top `5 zip codes` that show the highest potential `return on investment (ROI) in 5 years.`
* To determine the `5 least advisable cities` to invest in.
* To identify which `top 5 metropolitan areas` and states have the highest market performance in terms of price growth.
* To determine the best month to invest to maximize ROI.
* To show how factors like economic stability(recession) impact housing market performance.

#### Hypotheis testing
**Null Hypothesis (H0):** The variables do not help predict future real estate prices

**Alternative Hypothesis (H1):** The variables predict future real estate prices

## Data Understanding

### A Brief Description of the Dataset

This data represents median monthly housing sales prices for `265 zip codes in the USA`, over the period of `April 1996 through April 2018` as reported by Zillow.

Each row represents a unique zip code. Each record contains location information and median housing sales prices for each month.

There are `11568 rows and 272 variables`:

+ **RegionID:** Unique index
+ **RegionName:** Unique Zip Code
+ **City:** City in which the zip code is located
+ **State:** State in which the zip code is located
+ **Metro:** Metropolitan Area in which the zip code is located
+ **CountyName:** County in which the zip code is located
+ **SizeRank:** Numerical rank of size of zip code, ranked 1 through 14723
+ **1996-04 through 2018-04:** refers to the median housing sales values for `April 1996` through `April 2018`, that is `265 data points` of monthly data for each zip code

We shall check for the accuracy of our forecasts using `MSE (Mean Squared Error)`. This will provide us with the average error of our forecasts.


### Assumptions:
1. The zip codes are representative of the broader real estate market in the United States.
2. The real estate market conditions during this period were influenced by various economic and socio-political factors.
3. The data covers a combination of urban, suburban, and rural area.
4. The data doesn't reflect significant improvements or renovations made to properties over time that could increase or decrease prices.

### Reviewing and Inspecting the Data

In this step, we load, review, and examine the dataset further

In [1]:
# Import libraries
import numpy as np
import folium
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy import stats
from random import gauss as gs
import math
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas.plotting import autocorrelation_plot, lag_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
from matplotlib.pylab import rcParams
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#loading dataset and viewing first 5 rows
data = pd.read_csv('data/zillow_data.csv')

data.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
8,91940,77449,Katy,TX,Houston,Harris,9,95400.0,95600.0,95800.0,...,166800,167400,168400,169600,170900,172300,173300,174200,175400,176200
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500


In [3]:
data.tail(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
14713,59187,3765,Haverhill,NH,Claremont,Grafton,14714,80800.0,80100.0,79400.0,...,119800,120000,120800,121600,122600,123400,124400,125500,126600,127500
14714,94711,84781,Pine Valley,UT,St. George,Washington,14715,135900.0,136300.0,136600.0,...,241100,243200,244300,248900,254600,257600,258000,260600,264700,266800
14715,62556,12429,Esopus,NY,Kingston,Ulster,14716,78300.0,78300.0,78200.0,...,164200,166600,170000,171000,170700,171300,172400,173600,175800,177500
14716,99032,97028,Rhododendron,OR,Portland,Clackamas,14717,136200.0,136600.0,136800.0,...,332600,332900,335600,338900,340500,341000,342300,345000,348000,349300
14717,62697,12720,Bethel,NY,NaN,Sullivan,14718,62500.0,62600.0,62700.0,...,122200,122700,122300,122000,122200,122800,123200,123200,120700,117700
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400
14722,95851,89155,Mesquite,NV,Las Vegas,Clark,14723,176400.0,176300.0,176100.0,...,333800,336400,339700,343800,346800,348900,350400,353000,356000,357200


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [5]:
data.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)

In [6]:
# Examine the shape of the dataset
data.shape

(14723, 272)

In [7]:
# Viewing number of zipcodes in dataset and NAN values
print('Total number of zipcodes:', len(data))
data.iloc[:, :20].isna().sum()

Total number of zipcodes: 14723


RegionID         0
RegionName       0
City             0
State            0
Metro         1043
CountyName       0
SizeRank         0
1996-04       1039
1996-05       1039
1996-06       1039
1996-07       1039
1996-08       1039
1996-09       1039
1996-10       1039
1996-11       1039
1996-12       1039
1997-01       1039
1997-02       1039
1997-03       1039
1997-04       1039
dtype: int64

In [8]:
#Inspecting dataframe dtypes

data.iloc[:, :10].dtypes

RegionID        int64
RegionName      int64
City           object
State          object
Metro          object
CountyName     object
SizeRank        int64
1996-04       float64
1996-05       float64
1996-06       float64
dtype: object

In [9]:
# change 'RegionName' column to 'ZipCode' to avoid confusion
data.rename(columns={'RegionName': 'ZipCode'}, inplace=True)

data.ZipCode = data.ZipCode.astype('string')

#confirming the change
print(data.dtypes["ZipCode"])

string


In [10]:
# check for duplicates
data.duplicated().sum()

0

In [11]:
# all missing data because of the datatime appearing as columns
data.isnull().sum()

RegionID       0
ZipCode        0
City           0
State          0
Metro       1043
            ... 
2017-12        0
2018-01        0
2018-02        0
2018-03        0
2018-04        0
Length: 272, dtype: int64

## Exploratory Data Analysis(EDA) and Feature Engineering

### Missing Values

There are some missing values in the `Metro` and `dates` columns.

In [12]:
# Define a function to explore missing data
def missing_data(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data>0]
    return missing_data.to_frame()

# Apply missing_data function to the dataframe
missing_data(data)

,0
Metro,1043
1996-04,1039
1996-05,1039
1996-06,1039
1996-07,1039
...,...
2014-02,56
2014-03,56
2014-04,56
2014-05,56


- Missing values in the **Metro** column are replaced with **missing**.  
- Missing values in the dates columns are backfilled.

In [13]:
# replacing the missing values bin Metro with 'missing'
data.Metro.fillna('missing', inplace=True)

# interpolate missing values on date columns
data.interpolate(method="pad", inplace=True)

data.isna().sum().sum()

0

### Creating New Columns for Analysis

- In finance, the coefficient of variation allows investors to determine how much volatility, or risk, is assumed in comparison to the amount of return expected from investments.
- Ideally, if the coefficient of variation formula should result in a lower ratio of the standard deviation to mean return, then the better the risk-return trade-off.
- (**CONFIRM**)Therefore, in these next steps we are going to filter the data some more by calculating the CV value and only selecting values with in the company's risk factor (assume 60 percentile).

In [ ]:
# # calculating and creating a new column -ROI
# data['ROI'] = (data['2018-04']/ data['1996-04'])-1

# #calculating std to be used to find CV
# data["std"] = data.loc[:, "1996-04":"2018-04"].std(skipna=True, axis=1)

# #calculating mean to be used to find CV
# data["mean"] = data.loc[:, "1996-04":"2018-04"].mean(skipna=True, axis=1)

# # calculating and creating a new column - CV
# data["CV"] = data['std']/ data["mean"]

# data.columns

In [ ]:
# # List of 5-year periods start and end years
# periods = [(1998, 2003), (2003, 2008), (2008, 2013), (2013, 2018)]

# # Create a new DataFrame to store ROI values
# roi_data = pd.DataFrame()

# # Calculate ROI for each period and store in the new DataFrame
# for start_year, end_year in periods:
#     start_col = str(start_year) + '-04'  # Assuming April data is available for each year
#     end_col = str(end_year) + '-04'

#     roi_data['ROI_' + str(start_year) + '-' + str(end_year)] = (
#         (data[end_col] - data[start_col]) / data[start_col]
#     ) * 100

# # Display the new DataFrame with ROI values for each period
# print(roi_data)


### Select Top 10 States to invest in
As per this article by [Fit Small Business](https://fitsmallbusiness.com/best-and-worst-states-to-invest-in-real-estate/), the 10 states for the best yields in real estate are Georgia, Utah, Texas, North Carolina and Texas.
- This narrows down our number of Zip Codes to 4039.



In [14]:
# List of states to invest in
states_to_invest = ['GA', 'UT', 'TX', 'NC', 'NJ', 'TN', 'WA', 'DE', 'NE', 'FL']

# Filter data for the selected states
data_states = data[data['State'].isin(states_to_invest)].copy()

# Drop unnecessary columns
columns_to_drop = ['RegionID', 'SizeRank']
data_states.drop(columns_to_drop, axis=1, inplace=True)

# Reset the index
data_states.reset_index(drop=True, inplace=True)

print('Total Zipcodes in DataFrame:', len(data_states))
data_states.head()

Total Zipcodes in DataFrame: 4039


,ZipCode,City,State,Metro,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,75070,McKinney,TX,Dallas-Fort Worth,Collin,235700.0,236900.0,236700.0,235400.0,233300.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
1,77494,Katy,TX,Houston,Harris,210400.0,212200.0,212200.0,210700.0,208300.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
2,79936,El Paso,TX,El Paso,El Paso,77300.0,77300.0,77300.0,77300.0,77400.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
3,77084,Houston,TX,Houston,Harris,95000.0,95200.0,95400.0,95700.0,95900.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
4,77449,Katy,TX,Houston,Harris,95400.0,95600.0,95800.0,96100.0,96400.0,...,166800,167400,168400,169600,170900,172300,173300,174200,175400,176200


In [ ]:
#Check for zeros
data_states.describe()

### Calculating ROI and CV

Calculating the average price for the last five years to narrow down the data.

In [ ]:
# average price for the last 5 years (2013 to 2018)
data_states['yr_avg']=data_states.iloc[:,-60:].mean(skipna=True, axis=1)

#Calculate the 75% cutoff value
q_75 = data_states['yr_avg'].quantile(q=0.75)
print(f'Average Value 75% cutoff value: {round(q_75,2)}')

#Calculate the 25% cutoff value
q_25 = data_states['yr_avg'].quantile(q=0.25)
print(f'Average Value 25% cutoff value: {round(q_25,2)}')

#Get data frame with selected zipcodes.
states_avg = data_states[(data_states['yr_avg']<q_75) & (data_states['yr_avg']>q_25)]
print(f'Amount of zipcodes: {len(states_avg)}')

In [ ]:
states_avg.head()

In [ ]:
#Calculate historical return on investment
data_states['ROI'] = (data_states['yr_avg']/data_states['1996-04'])-1

## 5 year ROI
data_states['ROI_5yr'] = round((data_states['2018-04'] - data_states['2013-01'])/ data_states['2013-01'],4)

## 3 year ROI
data_states['ROI_3yr'] = round((data_states['2018-04'] - data_states['2015-01'])/ data_states['2015-01'],4)

#Calculate standard deviation of monthly values
data_states['std'] = data_states.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
data_states['mean'] = data_states.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)

#Calculate coefficient of variation
data_states['CV'] = data_states['std']/data_states['mean']

#Show calculated values
data_states[['State', 'ZipCode','std','mean','ROI','ROI_5yr', 'ROI_3yr', 'CV']].head()

In [ ]:
states_roi = data_states.groupby('State', group_keys=False).sum()['ROI']
states_roi.sort_values(ascending=False)

In [ ]:
#Define upper limit of CV according to risk profile.
cv_60 = data_states.CV.quantile(.6)
print(f'\nCV upper limit: {cv_60}')

In [ ]:
# Group states by highest ROIs within the firms risk profile.
states_roi_cv = data_states[data_states['CV'] < cv_60].sort_values('ROI', axis=0, ascending=False)
states_roi_cv.head()

In [ ]:
states_roi_cv.shape

In [ ]:
# No. of zipcodes per state
states_roi_cv['State'].value_counts()

In [ ]:
### not sure why
states_roi_cv.groupby('State').max()['ROI']

In [ ]:
states_roi_cv.isna().sum()

In [ ]:
states_roi_cv_grouped = states_roi_cv.groupby('State').sum()['ROI']
states_roi_cv_grouped.sort_values(ascending=False)

# max roi, least cv
target_states = list(states_roi_cv_grouped.sort_values(ascending=False).index)
target_states

In [ ]:
# Filter the DataFrame for the target states
states_combined = states_roi_cv[states_roi_cv['State'].isin(target_states)]

# Print the filtered DataFrame
states_combined.head()

In [ ]:
states_roi_cv.columns

In [ ]:
# creating a function that changes the dataframe structure from wide view to long view

def melt_df(data):
    melted = pd.melt(data, id_vars=['ZipCode', 'City', 'State', 'Metro',
                                    'CountyName', 'yr_avg', 'ROI', 'ROI_5yr',
                                    'ROI_3yr',	'std',	'mean',	'CV'],
                     var_name='Date')
    melted['Date'] = pd.to_datetime(melted['Date'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

states_melted = melt_df(states_combined)

states_melted.head()

In [ ]:
# Convert 'date' column to datetime format
states_melted['Date'] = pd.to_datetime(states_melted['Date'])

# Set the 'Date' column as index
states_melted.set_index('Date', inplace=True)

In [ ]:
# Sort the DataFrame by ROI in descending order
top_10_zipcodes = states_melted.sort_values(by='ROI', ascending=False)

# Print the top 10 zip codes by ROI
top_10_zipcodes

### **Converting to Time Series**

**Step 1: Changing the dataframe from wide view to long view**

In [ ]:
melted_data = states_melted.copy()

In [ ]:
# creating a function that changes the dataframe structure from wide view to long view

def melt_df(data):
    melted = pd.melt(data, id_vars=['ZipCode', 'City', 'State', 'Metro',
                                    'CountyName'], var_name='Date')
    melted_data['Date'] = pd.to_datetime(melted_data['Date'], format='%Y-%m-%d')
    melted = melted.dropna(subset=['value'])
    return melted

melted_data = melt_df(melted_data)

**Step 2: Converting the Date Column to DateTime Format and Setting it as the Index**

In [ ]:
# Convert 'date' column to datetime format
melted_data['Date'] = pd.to_datetime(melted_data['Date'])

# Set the 'Date' column as index
melted_data.set_index('Date', inplace=True)

In [ ]:
melted_data.head()

In [ ]:
print("Columns in melted_data:", melted_data.columns)
print()
print("Columns in states_combined:", states_combined.columns)

In [ ]:
# Join the dataframes based on common columns
final_df = melted_data.merge(states_combined, on=['ZipCode', 'City', 'State', 'Metro', 'CountyName'])

# Set the 'Date' column as the index
final_df.set_index('Date', inplace=True)

# Print the resulting DataFrame
final_df.head()

### **Narrowing Down to a Few Zip Codes from the Data**
- The data is quite large.
- Therefore, the analysis will filter out the first 1000 zip codes per state based on median prices (`value`).

In [ ]:
# # Group by 'State' and get the top 1000 zip codes for each state based on median prices
# top_zipcodes = (melted_data.sort_values(['State', 'value'], ascending=[True, False])
#                 .groupby('State')
#                 .head(1000))

# # # Create a new DataFrame with the selected data
# top_zipcodes_df = pd.DataFrame({
#     # 'Date': top_zipcodes['Date'],
#     'State': top_zipcodes['State'],
#     'County': top_zipcodes['CountyName'],
#     'Metro': top_zipcodes['Metro'],
#     'City': top_zipcodes['City'],
#     'ZipCode': top_zipcodes['ZipCode'],
#     'SizeRank': top_zipcodes['SizeRank'],
#     'Value': top_zipcodes['value']

# })
# # # Display the result
# top_zipcodes_df.head()

In [ ]:
top_zipcodes_df.reset_index()

In [ ]:
top_zipcodes_df.index.name

In [ ]:
# List of 5-year periods start and end years
periods = [(1998, 2003), (2003, 2008), (2008, 2013), (2013, 2018)]

# Create an empty DataFrame to store ROI values
roi_data = pd.DataFrame()

# Iterate over each zip code
for zip_code in top_zipcodes_df['ZipCode']:
    zip_roi_data = {'ZipCode': zip_code}

    for start_year, end_year in periods:
        start_col = str(start_year) + '-04'  # Assuming April data is available for each year
        end_col = str(end_year) + '-04'

        start_value = top_zipcodes_df[top_zipcodes_df['ZipCode'] == zip_code][start_col].values[1]
        end_value = top_zipcodes_df[top_zipcodes_df['ZipCode'] == zip_code][end_col].values[1]

        roi = ((end_value - start_value) / start_value) * 100
        col_name = f'ROI_{start_year}-{end_year}'
        zip_roi_data[col_name] = roi

    # Append the data for the current zip code to the roi_data DataFrame
    roi_data = roi_data.append(zip_roi_data, ignore_index=True)

# Display the new DataFrame with ROI values for each period
print(roi_data)


In [ ]:
# confirming the new columns are added correctly
melted_data.info()

In [ ]:
# ROI 5-years


In [ ]:
# Assuming your datetime column is named 'Date' and it's the index of your melted_data DataFrame
melted_data['Year'] = melted_data.index.year

# Define a function to create 5-year bands
def create_5_year_band(year):
    return f"{year // 5 * 5}-{year // 5 * 5 + 4}"

# Apply the function to the 'Year' column to create the 5-year bands
melted_data['5_Year_Band'] = melted_data['Year'].apply(create_5_year_band)
melted_data.head()


In [ ]:
top_zipcodes_df.index.name

In [ ]:
top_zipcodes_df.info()

- The data now has 51000 records, i.e. 1000 zipcodes from 51 states.

In [ ]:
top_zipcodes_df.columns

In [ ]:
top_zipcodes_df.describe()

### **5-year Return on Investment**

In [ ]:
# # Create a new 'year' column based on the year from the datetime index
# top_zipcodes_df['Year'] = top_zipcodes_df.index.year

# # Define a function to group dates into 5-year periods
# def create_5_year_periods(year):
#     return f"{year}-{year+4}"

# # Apply the function to create the '5-year-periods' column
# top_zipcodes_df['5-year-periods'] = top_zipcodes_df['Year'].apply(create_5_year_periods)

# top_zipcodes_df.tail()

#### **Calculate Mean ROI for each 5-year period**

In [ ]:
mean_roi_by_period = top_zipcodes_df.groupby('5_Year_Band')['ROI'].mean()

# Plotting the data
plt.figure(figsize=(10, 6))
mean_roi_by_period.plot(kind='bar', color='blue')
plt.title('Mean ROI by 5-Year Period')
plt.xlabel('5-Year Period')
plt.ylabel('Mean ROI')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### **Determine the top 5 zip codes that show the highest potential return on investment (ROI)**

In [ ]:
# Create a DataFrame of the top 5 zip codes
top_zipcodes_ROI = pd.DataFrame(top_zipcodes_df.groupby("ZipCode")["ROI"].mean().sort_values(ascending=False).head(5))
print(top_zipcodes_ROI)

# Create a bar chart of the average ROI
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
top_zipcodes_ROI.plot(kind='bar', ax=ax, color="blue")

# Set the title, labels, and ticks
plt.title("Zipcodes with the Highest ROI")
ax.set_ylabel("Average ROI");
ax.set_xlabel("Zipcodes")
# plt.xticks(fontsize= 18)

# Show the bar chart
plt.show()

In [ ]:
#top 5 zipcodes in terms of ROI
top_5_zipcodes_ROI =['10021', '31561', '10011', '11231', '10014']

top5_roi_df = top_zipcodes_df[top_zipcodes_df['ZipCode'].isin(top_5_zipcodes_ROI)]
top5_roi_df.head()

In [ ]:
# Create a DataFrame of the top 5 zip codes
top_zipcodes_CV = pd.DataFrame(top_zipcodes_df.groupby("ZipCode")["CV"].mean().sort_values(ascending=False).head(5))
print(top_zipcodes_CV)

# Create a bar chart of the average CV
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
top_zipcodes_CV.plot(kind='bar', ax=ax, color="red")

# Set the title, labels, and ticks
plt.title("Zipcodes with the Highest CV")
ax.set_ylabel("Average CV");
ax.set_xlabel("Zipcodes")
# plt.xticks(fontsize= 18)

# Show the bar chart
plt.show()

In [ ]:
#top 5 zipcodes in terms of CV
top_5_zipcodes_CV =['56041', '2116', '16102', '43013', '24054']

top5_cv_df = top_zipcodes_df[top_zipcodes_df['ZipCode'].isin(top_5_zipcodes_CV)]
top5_cv_df.head()

### **Determine the 5 Least Advisable Cities to Invest in**

In [ ]:
# Create a DataFrame of the bottom 5 zip codes
bottom_zipcodes_ROI = pd.DataFrame(top_zipcodes_df.groupby("ZipCode")["ROI"].mean().sort_values(ascending=False).tail(5))
print(bottom_zipcodes_ROI)

# Create a bar chart of the average ROI
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
bottom_zipcodes_ROI.plot(kind='bar', ax=ax, color="blue")

# Set the title, labels, and ticks
plt.title("Zipcodes with the Lowest ROI")
ax.set_ylabel("Average ROI");
ax.set_xlabel("Zipcodes")
# plt.xticks(fontsize= 18)

# Show the bar chart
plt.show()

In [ ]:
# bottom 5 zipcodes in terms of ROI
bottom_5_zipcodes_ROI =['67473', '71075', '24054', '28039', '16102']

bottom5_roi_df = top_zipcodes_df[top_zipcodes_df['ZipCode'].isin(bottom_5_zipcodes_ROI)]
bottom5_df.head()

In [ ]:
# Create a DataFrame of the bottom 5 zip codes
bottom_zipcodes_CV = pd.DataFrame(top_zipcodes_df.groupby("ZipCode")["CV"].mean().sort_values(ascending=False).tail(5))
print(bottom_zipcodes_CV)

# Create a bar chart of the average ROI
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
bottom_zipcodes_CV.plot(kind='bar', ax=ax, color="red")

# Set the title, labels, and ticks
plt.title("Zipcodes with the Lowest CV")
ax.set_ylabel("Average ROI");
ax.set_xlabel("Zipcodes")
# plt.xticks(fontsize= 18)

# Show the bar chart
plt.show()

In [ ]:
# bottom 5 zipcodes in terms of CV
bottom_5_zipcodes_CV =['38139', '74137', '46037', '46077', '44040']

bottom5_cv_df = top_zipcodes_df[top_zipcodes_df['ZipCode'].isin(bottom_5_zipcodes_CV)]
bottom5_cv_df.head()

### **Which are the Top 5 Metro Areas and States with the Highest Market Performance in Terms of Price Growth** (correct graph)

In [ ]:
# Group the data by metro area and state and calculate the mean ROI
top_metro_areas_df = top_zipcodes_df.groupby(["Metro", "State"])["ROI"].mean()

# Sort by mean ROI
top_metro_areas_df = top_metro_areas_df.sort_values(ascending=True)

# Get the top 5 metropolitan areas
top_5_metro_areas = top_metro_areas_df.index[:5]

# Print the top 5 metropolitan areas
print("Top 5 Metropolitan Areas with the Highest Market Performance in Terms of Price Growth:")
for metro_area in top_5_metro_areas:
    print(metro_area)

# Plot a horizontal bar graph
plt.figure(figsize=(10, 6))
top_metro_areas_df.loc[top_5_metro_areas].plot(kind='barh', color='blue')
plt.xlabel('Mean ROI')
plt.ylabel('Metro Area, State')
plt.title('Top 5 Metropolitan Areas with the Highest Market Performance')
plt.tight_layout()
plt.show()

In [ ]:
# Group the data by state and calculate the mean ROI
top_cities_df = top_zipcodes_df.groupby(["City", "State"])["ROI"].mean()

# Sort by mean ROI
top_cities_df = top_cities_df.sort_values(ascending=False)

# Get the top 5 states
top_cities_df = top_cities_df.index[:5]

# Print the top 5 states
print("Top 5 Cities with the Highest Market Performance in Terms of Price Growth:")
for city in top_cities_df:
    print(city)

### **Determine the best month to invest to maximize ROI**

In [ ]:
# Group the data by month and calculate the mean ROI
monthly_roi_df = top_zipcodes_df.groupby("Date")["ROI"].mean()

# Sort by mean ROI
monthly_roi_df = monthly_roi_df.sort_values(ascending=False)

# Get the month with the highest mean ROI
best_month = monthly_roi_df.index[0]

# Print the best month
print("The best month to invest to maximize ROI is:", best_month)

In [ ]:
monthly_roi_df.head()

In [ ]:
monthly_data = top_zipcodes_df['Value'].resample('MS').mean()
monthly_data = monthly_data.fillna(monthly_data.bfill())
monthly_data.plot(figsize=(8,5))
plt.title('Average Home Value By Month')
plt.ylabel('Value')
plt.show()
print("Earliest Dates")
print(monthly_data.head())
print()
print("Most Recent")
print(monthly_data.tail())

In [ ]:
monthly_data['2012':].plot(figsize=(8,5))

## Modeling
- To begin the modelling, the data will be checked for `trend`, `seasonality` and `residuals`.

### Checking for Trend (correct this)

In [ ]:
#checking for trends in the dataset

# Calculate mean ROI for each ZipCode and sort in descending order
top_zipcodes_by_mean_roi = top_zipcodes_df.groupby("ZipCode")["ROI"].mean().sort_values(ascending=False).head(20)

for zip_code, region_data in top_zipcodes_by_mean_roi.groupby("ZipCode"):
  zipcode_data = top_zipcodes_df[top_zipcodes_df["ZipCode"] == zip_code]
  values = zipcode_data['ROI'].mean()  # Select the mean 'ROI'
  plt.plot(values, label=zip_code)  # Plot the values with zip code as label

plt.xlabel('Time')
plt.ylabel('ROI')
plt.title('Line Plot by Zip Code')
plt.legend()
plt.show()

In [ ]:
def seasonal_decomposition(df):
    decomposition = seasonal_decompose(df)

    # Gather the trend, seasonality, and residuals
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Plot gathered statistics
    plt.figure(figsize=(8,8))
    plt.subplot(411)
    plt.plot(df, label='Original', color='blue')
    #plt.plot(ts, label='Original', color='blue')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='blue')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality', color='blue')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residuals', color='blue')
    plt.legend(loc='best')
    plt.tight_layout()

### Check for Seasonality

In [ ]:
seasonal_decompose(top_zipcodes_df["ROI"])

### Check for Stationarity

In [ ]:
# Extract the 'ROI' column for analysis
roi_series = top_zipcodes_df['ROI']

# Apply the ADF test
result = adfuller(roi_series)

# Print the ADF test results
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Critical Values:", result[4])
print("Is Stationary:", "No" if result[1] > 0.05 else "Yes")


- The series `**is stationary**` because the p-value is less than 0.05.
- There will be no need to difference.

In [ ]:
# Checking by State
grouped = top_zipcodes_df.groupby('State')

# For each group (State) apply ADF test
for state, group in grouped:

    # Extract the ROI values for the current state
    roi_values = group['ROI']

    # Performing Augmented Dickey Fuller test
    result = adfuller(roi_values)

    # Print State Name, Test Statistics and p-value
    print(f"State: {state}")
    print(f"Test Statistics: {result[0]}")
    print(f"p-value: {result[1]}")

    # Determine if the data is stationary
    if result[1] < 0.05:
        print("Data is stationary")
    else:
        print("Data is not stationary")

    print("\n")


ARIMA

In [ ]:
# top_zipcodes_df to csv
csv_file_path = 'top_zipcodes_data.csv'
top_zipcodes_df.to_csv(csv_file_path, index=False)